In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pyexpat import features
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. Wczytywanie danych

In [ ]:
file_path = "data.txt"
data = pd.read_csv(file_path, delimiter= "\t")  # Pomijamy opisowe linie
# Wyświetlenie pierwszych wierszy
print(data.head())


# Wyświetlenie wszystkich wykresów punktowych oddzielnie

In [ ]:
variables = data.columns.tolist()
#ariables.remove("Pct.BF")  # Usuwamy "Pct.BF", ponieważ jest zmienną zależną

# Tworzenie wykresów punktowych
for variable in variables:
    plt.figure(figsize=(8, 6))
    plt.scatter(data[variable], data["Pct.BF"], alpha=0.7)
    plt.title(f"Wykres punktowy: Pct.BF vs {variable}")
    plt.xlabel(variable)
    plt.ylabel("Pct.BF")
    plt.grid(True)
    plt.show()  # Wyświetlenie każdego wykresu osobno

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(12, 12))
i = 0

for i, (name, X) in enumerate(data.items()):
    row = i // 4
    col = i % 4
    
    axs[row][col].scatter(X, data['Pct.BF'])
    axs[row][col].set_xlabel(name)
    axs[row][col].set_ylabel('Pct.BF')

fig.tight_layout()
fig.show()

In [ ]:
# Obliczenie macierzy korelacji
correlation_matrix = data.corr()

# Wyświetlenie korelacji z "Pct.BF"
correlations_with_bodyfat = correlation_matrix["Pct.BF"].sort_values(ascending=False)
print("Korelacje ze zmienną 'Pct.BF':\n", correlations_with_bodyfat)


# model ze wszystkimi zmiennymi:

In [ ]:
# Lista wszystkich zmiennych predykcyjnych (pomijając zmienną zależną "Pct.BF")
predictors = [col for col in data.columns if col != "Pct.BF"]

# Przygotowanie danych
X = sm.add_constant(data[predictors])  # Wszystkie zmienne predykcyjne + stała
y = data["Pct.BF"]  # Zmienna zależna

# Tworzenie pełnego modelu
model = sm.OLS(y, X).fit()

# Raport modelu regresji (wyniki statystyczne)
print(model.summary())


# FORWARD SELECTION

In [ ]:
#Forward selection na p-value
X = data.drop(columns=["Pct.BF"])
y = data["Pct.BF"]
def forward_selection_pvalue(X, y, significance_level=0.05, verbose=True):
   
    initial_list = []  # Lista wybranych zmiennych (na początku pusta)
    remaining_features = list(X.columns)  # Wszystkie dostępne zmienne
    step = 0
    best_model = None

    while len(remaining_features) > 0:
        p_values = pd.Series(index=remaining_features)
        for feature in remaining_features:
            temp_features = initial_list + [feature]
            model = sm.OLS(y, sm.add_constant(X[temp_features])).fit()
            p_values[feature] = model.pvalues[feature]

        min_p_value = p_values.min()
        best_feature = p_values.idxmin()

        if min_p_value < significance_level:
            step += 1
            initial_list.append(best_feature)
            remaining_features.remove(best_feature)
            best_model = sm.OLS(y, sm.add_constant(X[initial_list])).fit()
            if verbose:
                print(f"Krok {step}: Added {best_feature}, p-value = {min_p_value}")
        else:
            break

    return best_model


In [ ]:
# Regresja krokowa w przód oparta na p-wartościach
# Standaryzacja predyktorów
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_scaled = sm.add_constant(X_scaled)  # Dodawanie stałej 

print("\nKroki regresji krokowej w przód na podstawie p-wartości:")
final_model_forward = forward_selection_pvalue(X_scaled, y)


print("\nFinal Model Coefficients (Forward Selection):")
print(final_model_forward.params)

print("\nPodsumowanie finalnego modelu:")
print(final_model_forward.summary())


# LASSO

In [ ]:
# Skalowanie danych
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Selekcja zmiennych (jeśli jest już gotowa lista 'selected_features')
# Jeśli nie masz wybranych zmiennych, użyj wszystkich
selected_features = X.columns.tolist()
X_train_selected = X_train_scaled[:, [X.columns.get_loc(f) for f in selected_features]]
X_test_selected = X_test_scaled[:, [X.columns.get_loc(f) for f in selected_features]]

# LASSO z walidacją krzyżową (5-krotny podział)
lasso_cv = LassoCV(cv=5, random_state=42).fit(X_train_selected, y_train)
optimal_alpha = lasso_cv.alpha_
print('Optymalne alpha LASSO: ', optimal_alpha)
print("Współczynniki LASSO:")
print(pd.Series(lasso_cv.coef_, index=selected_features))

# Ocena na danych testowych
y_pred = lasso_cv.predict(X_test_selected)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Błąd MSE na danych testowych:", mse)
print("Współczynnik determinacji (R²):", r2)

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("Rzeczywiste wartości (Pct.BF)")
plt.ylabel("Przewidywane wartości (Pct.BF)")
plt.title("Rzeczywiste vs Przewidywane")
plt.show()

# Backward SELECTION na p-value

In [ ]:
def backward_selection_pvalue(X, y, significance_level=0.05, verbose=True):
    
    initial_list = X.columns.tolist()
    step = 0
    while True:
        model = sm.OLS(y, X).fit()
        p_values = model.pvalues[1:]  # Pomijamy stałą 
        max_p_value = p_values.max()
        
        if max_p_value > significance_level:
            step += 1
            worst_feature = p_values.idxmax()
            if verbose:
                print(f"Krok {step}: Removed {worst_feature}, p-value = {max_p_value}")
            X = X.drop(columns=[worst_feature])
        else:
            break
    
    return model


In [ ]:
# Standaryzacja predyktorów
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_scaled = sm.add_constant(X_scaled)  # Dodawanie stałą (intercept)

# Regresja wsteczna oparta na p-wartościach
print("\nKroki regresji wstecznej na podstawie p-wartości:")
final_model = backward_selection_pvalue(X_scaled, y)

print("\nFinal Model Coefficients:")
print(final_model.params)


print("\nPodsumowanie finalnego modelu:")
print(final_model.summary())


# Nowy model - predykcja wagi

In [ ]:
X = data.drop(columns=["Weight"])
y = data["Weight"]

#Backward Rgression:
def backward_selection_weight(X, y, significance_level=0.05, verbose=True):
    initial_list = X.columns.tolist()
    step = 0
    while True:
        model = sm.OLS(y, X).fit()
        p_values = model.pvalues[1:]  # Pomijamy stałą
        max_p_value = p_values.max()
        
        if max_p_value > significance_level:
            step += 1
            worst_feature = p_values.idxmax()
            if verbose:
                print(f"Krok {step}: Removed {worst_feature}, p-value = {max_p_value}")
            X = X.drop(columns=[worst_feature])
        else:
            break
    
    return model


In [ ]:
# Standaryzacja predyktorów
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_scaled = sm.add_constant(X_scaled)  # Dodawanie stałą (intercept)

# Regresja wsteczna oparta na p-wartościach
print("\nKroki regresji wstecznej na podstawie p-wartości:")
final_model = backward_selection_pvalue(X_scaled, y)

print("\nFinal Model Coefficients:")
print(final_model.params)


print("\nPodsumowanie finalnego modelu:")
print(final_model.summary())